## Imports

In [1]:
import pandas as pd
import folium
import os

## Data Read-in

In [2]:
df = pd.read_csv('Eviction_Notices.csv')

## Make Map

In [3]:
df['File Date'] = pd.to_datetime(df['File Date'])

In [4]:
df = df[df['File Date'] >= '2023-01-01']

In [5]:
df['Location'].isna().value_counts()

Location
False    1380
True       27
Name: count, dtype: int64

In [6]:
df = df.dropna(subset='Location')

In [7]:
def parse_location(location_str):
    # Remove the parentheses and split the string into latitude and longitude
    lat, lon = map(float, location_str[1:-1].split(','))
    return pd.Series([lat, lon], index=['latitude', 'longitude'])

# Apply the function to the 'location' column and join the result with the original DataFrame
coordinates = df['Location'].apply(parse_location)
df = df.join(coordinates)

In [9]:
pd.set_option('display.max_columns',None)

In [16]:
df['True_Count'] = df[['Non Payment', 'Breach',
       'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development']].sum(axis=1)

In [19]:
df['Eviction_Reason'] = df[['Non Payment', 'Breach',
       'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development']].idxmax(axis=1)

In [21]:
df['Eviction_Reason'].value_counts()

Eviction_Reason
Nuisance                460
Capital Improvement     350
Non Payment             216
Breach                  163
Owner Move In            95
Ellis Act WithDrawal     37
Roommate Same Unit       19
Other Cause              16
Unapproved Subtenant     15
Late Payments             5
Access Denial             4
Name: count, dtype: int64

In [23]:
df.columns

Index(['Eviction ID', 'Address', 'City', 'State',
       'Eviction Notice Source Zipcode', 'File Date', 'Non Payment', 'Breach',
       'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development',
       'Good Samaritan Ends', 'Constraints Date', 'Supervisor District',
       'Neighborhoods - Analysis Boundaries', 'Location', 'Shape',
       'SF Find Neighborhoods', 'Current Police Districts',
       'Current Supervisor Districts', 'Analysis Neighborhoods',
       'DELETE - Neighborhoods', 'DELETE - Police Districts',
       'DELETE - Supervisor Districts', 'DELETE - Fire Prevention Districts',
       'DELETE - Zip Codes', 'CBD, BID and GBD Boundaries as of 2017',
       'Central Market/Tenderloin Boundary', 'Areas of Vu

In [25]:
def popup_html(row):
    address = row['Address']
    eviction_reason = row['Eviction_Reason']
    latitude = row['latitude']
    longitude = row['longitude']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Eviction Reason: </strong>{}'''.format(eviction_reason) + '''<br>
    <strong>Latitude: </strong>{}'''.format(latitude) + '''<br>
    <strong>Longitude: </strong>{}'''.format(longitude) + '''<br>
    </html>
    '''
    return html

In [24]:
# Create a map object
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)
# Add markers to the map
for i, row in df.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']]).add_to(m)
    
# Save it to an HTML file
m.save('index.html')
# Display the map
m

In [44]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/SF_Evictions2023
